<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Define-Problem" data-toc-modified-id="Define-Problem-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Define Problem</a></span></li><li><span><a href="#Parameter" data-toc-modified-id="Parameter-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Parameter</a></span></li><li><span><a href="#Decision-Variables" data-toc-modified-id="Decision-Variables-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Decision Variables</a></span></li><li><span><a href="#Objective" data-toc-modified-id="Objective-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Objective</a></span></li><li><span><a href="#Contraints" data-toc-modified-id="Contraints-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Contraints</a></span></li><li><span><a href="#Solve-the-Problem" data-toc-modified-id="Solve-the-Problem-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Solve the Problem</a></span></li></ul></div>

# Define Problem

In [1]:
!pip install pulp
from pulp import *
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Matching Employees", LpMaximize)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


# Parameter

In [4]:
import numpy as np 
import pandas as pd
np.random.seed(0)

c = np.random.randint(0,10, (8,8))
np.fill_diagonal(c,0)

c

array([[0, 0, 3, 3, 7, 9, 3, 5],
       [2, 0, 7, 6, 8, 8, 1, 6],
       [7, 7, 0, 1, 5, 9, 8, 9],
       [4, 3, 0, 0, 5, 0, 2, 3],
       [8, 1, 3, 3, 0, 7, 0, 1],
       [9, 9, 0, 4, 7, 0, 2, 7],
       [2, 0, 0, 4, 5, 5, 0, 8],
       [4, 1, 4, 9, 8, 1, 1, 0]])

In [2]:
employees = range(8)
group = 4

# Decision Variables

\begin{equation} \label{y_var_def}
    y_{i,j} = \left \{ \begin{array}{ll}
      1 & \mbox{if employee $i$ is paired with employee $j$}\\
      0 & \mbox{otherwise.} \end{array} \right.
  \end{equation}

In [3]:
#Define the variables
y = LpVariable.dicts("pair", [(i,j) for i in employees for j in employees] ,cat='Binary')

In [5]:
names = ['Ben','Kate','Thinh','Jorge','Alfredo','Francisco','Olivia','Chris']
match_info = pd.DataFrame(c, index=names, columns=names)

match_info

,Ben,Kate,Thinh,Jorge,Alfredo,Francisco,Olivia,Chris
Ben,0,0,3,3,7,9,3,5
Kate,2,0,7,6,8,8,1,6
Thinh,7,7,0,1,5,9,8,9
Jorge,4,3,0,0,5,0,2,3
Alfredo,8,1,3,3,0,7,0,1
Francisco,9,9,0,4,7,0,2,7
Olivia,2,0,0,4,5,5,0,8
Chris,4,1,4,9,8,1,1,0


# Objective

Maximize the preference scores between employees $i$ and $j$

Maximize $\sum_{j=0}^n\sum_{i=0}^n (c_{i,j}+c_{j,i}) \cdot y_{i,j}$

In [7]:
prob += lpSum([(c[i][j] + c[j][i]) * y[(i,j)] for i in employees for j in employees])

# Contraints

Employee $i$ is paired with no more than one employee $j$

\begin{equation}
\sum_{i} y_{i,j}\leq 1 \; \forall j \;\in employees
\end{equation}

Employee $j$ is paired with no more than one employee $i$

\begin{equation}
\sum_{i} y_{j,i} \leq 1 \; \forall j \;\in employees
\end{equation}

Pairing between employee $i$ and $j$ also means to pair between employee $j$ and $i$

\begin{equation}
y_{i,j} + y_{j,i} \leq 1 \; \forall i,j \;\in employees
\end{equation}

There is a total of 4 pairs

\begin{equation}
\sum_j\sum_{i} y_{i,j} = 4
\end{equation}

In [8]:
for i in employees:
    prob += lpSum(y[(i,j)] for j in employees) <= 1
    prob += lpSum(y[(j,i)] for j in employees) <= 1
    prob += lpSum(y[(i,j)] for j in employees)+ lpSum(y[(j,i)] for j in employees) <= 1


prob += lpSum(y[(i,j)] for i in employees for j in employees) == 4

# Solve the Problem

In [9]:
prob.solve()

1

In [14]:
print("Finish matching!\n")
for i in employees:
    for j in employees:
        if y[(i,j)].varValue == 1:
            print('{} and {} with preference score {} and {}. Total score: {}'.format(names[i],names[j],c[i,j], c[j,i], c[i,j] +c[j,i]))

Finish matching!

Ben and Alfredo with preference score 7 and 8. Total score: 15
Kate and Francisco with preference score 8 and 9. Total score: 17
Jorge and Chris with preference score 3 and 9. Total score: 12
Olivia and Thinh with preference score 0 and 8. Total score: 8


In [12]:
pulp.value(prob.objective)

26.0

In [13]:
pulp.LpStatus[prob.status]

'Optimal'